<div>
    <img src="http://www.ient.rwth-aachen.de/cms/uploads/images/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

## Teil 3: Klassifikation mit Neuronalen Netzen
In diesem Teil des Versuchs sollen Sie einmal selbst probieren, ein neuronales Netz, das handgeschriebene Ziffern erkennt, zu trainieren.
Dazu wird hier das MINST-Datenset verwendet. Die Architektur des Netzes sowie die Fehlerfunktion sind ebenfalls vorgegeben. Es ist Ihre Aufgabe eine geeignete Lernrate und Batchgröße zu finden so, dass die Klassifizierung mithilfe des fertigen Netzes möglichst gut funktioniert. 

Wie wirken sich die Lernrate und die Batchgröße auf das Training aus?

Wenn Sie die nachfolgende Box ausgeführt haben, erscheint eine GUI, mit der Sie das Netz trainieren können. Sobald das Training für mindestens 5 Batches gelaufen ist, können Sie sich auch einige Auswertungen und Zwischenergebnisse zu ihrem Netz ansehen. Um eine neue Lernrate oder Batchgröße auszuwählen, müssen Sie das Training einmal stoppen und mit den geänderten Werten neu beginnen. 
Wenn Sie denken, eine gute Variante trainiert zu haben, drücken Sie auf pause und führen den zweiten Codeblock aus. Dort können Sie selbst Ziffern in das schwarze Feld malen und das trainierte Netz darauf anwenden.

Nachfolgend finden Sie ein Video, dass Sie noch einmal kurz in die Funktion neuronaler Netze in der Bildverarbeitung einführt.

In [ ]:
from IPython.display import Video
Video("http://www.ient.rwth-aachen.de/services/PTI-Videos/V6_T3.mp4", width=480, height=270 )

In [ ]:
# Copyright 2020 Institut für Nachrichtentechnik, RWTH Aachen University
%matplotlib widget

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # weird OMP issue: it seems to be linked double, by Anaconda (used then by matplotlib)

# Imports
import IPython
from IPython.display import display
import ipywidgets as widgets

import time, warnings
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backend_bases import MouseButton
import threading

# RWTH imports
import rwth_nb.misc.feedback as rwth_feedback

# Tensorflow # Migrate Tf1 to Tf2: https://www.tensorflow.org/guide/migrate
from tensorflow.keras.datasets import mnist
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] ='3' # Mute TensorFlow backend

# Prepare threading
ipython = IPython.get_ipython()
ioloop = ipython.kernel.io_loop # The IOloop is shared

class MnistCnn:
    """Train MNIST with CNN."""
    
    def __init__(self):
        """Constructor"""
        
        # Prepare Data
        self.prepare_data()
             
    
    #======== PREPARE DATASET ========#
    def prepare_data(self):
        (self.image_train, self.label_train), (self.image_test, self.label_test) = mnist.load_data()
        self.image_train = self.image_train.reshape(-1, 28,28,1)
        self.image_test = self.image_test.reshape(-1, 28,28,1)
        self.label_train = to_categorical(self.label_train, 10)
        self.label_test = to_categorical(self.label_test, 10)
        self.image_train = self.image_train.astype('float32')
        self.image_test = self.image_test.astype('float32')
        self.image_train /= 255
        self.image_test /= 255

    
    #======== DECLARE NN WRAPPERS ========#
    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(self, shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(self, x, W, name=None):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME',name = name)

    def max_pool_2d(self, x, name):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME',name = name)
    def get_batch(self, iter_, size, trainFeatures, trainLabels):
        start_ = (iter_*size) % 42000
        return trainFeatures[start_ : start_ + size], trainLabels[start_ : start_ + size]

    #======== CREATE NETWORK FOR TRAINING ========#
    def conv_net(self,img_input, weights,bias):  

        # Layer 1: Convolutional Layer
        conv1 = self.conv2d(img_input, weights['wc1'])
        conv1 = tf.nn.relu(conv1 + bias['wc1'])
        conv1 = self.max_pool_2d(conv1, name='Conv1')

        # Layer 2: Convolutional Layer
        conv2 = self.conv2d(conv1, weights['wc2'])
        conv2 = tf.nn.relu(conv2 + bias['wc2'])
        conv2 = self.max_pool_2d(conv2, name='Conv2')

        # Layer 3: Fully Connected
        fc1 = tf.reshape(conv2, shape=[-1,7*7*16])
        out = tf.add(tf.matmul(fc1, weights['out']),bias['out'],name='net_out')
        smax = tf.nn.softmax(out, name='smax')

        return out  
    
    #======== TRAIN NETWORK ========#
    def train_model(self):
        self.init_model()
        for _ in range(self.epochs):
            for i in range(self.training_iters):
                self.train_model_step(i)

    def init_model(self, lr, batch_size, epochs):
        tf.reset_default_graph() # TODO: correct position here?
        
        self.i = -1
        self.j = -1
        self.train_loss = []
        self.valid_loss = []
        self.train_accuracy = []
        self.test_accuracy = []
        self.layer = []
        self.decisions = []
        self.lr = lr
        self.batch_size = batch_size
        self.epochs = epochs
        self.training_iters = int(self.image_train.shape[0]/self.batch_size)
        
        #======== DEFINING MODEL PARAMETERS ========#
        self.w_init = tf.random_normal_initializer()
        self.weights = { #Dictionary for different Weight values
            'wc1': tf.Variable(name = 'W0', initial_value=self.w_init(shape=(5,5,1,8))), #72 Parameter
            'wc2': tf.Variable(name = 'W1', initial_value=self.w_init(shape=(5,5,8,16))), #1152 Parameter
            'out': tf.Variable(name = 'W3', initial_value=self.w_init(shape=(7*7*16,10))) 
        }
        self.bias = { #Dictionary for different Bias values
            'wc1': tf.Variable(tf.constant(0.1, tf.float32, [8])),
            'wc2': tf.Variable(tf.constant(0.1, tf.float32, [16])),
            'out': tf.Variable(tf.constant(0.1, tf.float32, [10]))
        }
        self.x = tf.placeholder(tf.float32, [None, 28, 28,1], name='input')
        self.y = tf.placeholder(tf.float32, [None, 10], name = 'labels')

        
        #======== DEFINE NETWORK CALCULATIONS ========#
        self.nn = self.conv_net(self.x, self.weights,self.bias)
        self.correct_prediction = tf.equal(tf.argmax(self.nn, 1), tf.argmax(self.y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.nn, labels=self.y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.cost)
        
        #======== GET NETWORK LAYER OUTPUTS ========#
        self.C1 = tf.get_default_graph().get_tensor_by_name('Conv1:0')
        self.C2 = tf.get_default_graph().get_tensor_by_name('Conv2:0')
        self.out = tf.get_default_graph().get_tensor_by_name('net_out:0')
        self.smax = tf.get_default_graph().get_tensor_by_name('smax:0')   
        
        session_conf = tf.ConfigProto(intra_op_parallelism_threads=2,inter_op_parallelism_threads=2)
        self.session = tf.Session(config=session_conf)
        self.session.run(tf.global_variables_initializer())
        
    def train_model_step(self, i, detail=False):
        # Fetch training data (batch)
        batch_x, batch_y = self.get_batch(i, self.batch_size, self.image_train, self.label_train)
        
        # Actual training step
        if not detail:
            opt, loss, acc = self.session.run(
                [self.optimizer, self.cost, self.accuracy], feed_dict={'input:0': batch_x, 'labels:0': batch_y})
        else: # detailled step with a lot of extra computations
            opt, loss, acc, layer1, layer2, net_out, decision, weights1, weights2, weights3 = self.session.run(
                [self.optimizer, self.cost, self.accuracy, self.C1, self.C2, self.out, self.smax, 
                 self.weights['wc1'], self.weights['wc2'],self.weights['out']],
                feed_dict={'input:0': batch_x, 'labels:0': batch_y})
        
            self.layers = [np.squeeze(batch_x[0]),
                     conv_to_img(layer1[0],14,14,8,1,8),
                     conv_to_img(layer2[0],7,7,16,2,8),
                     array_to_img(net_out[0]),
                     array_to_img(decision[0]),
                     np.squeeze(batch_x[0]),
                     weights_to_img(weights1),
                     weights_to_img(weights2)
                    ]
        
        # Evaluation on test set
        test_acc, valid_loss, decisions = self.session.run([self.accuracy, self.cost, self.out], feed_dict={self.x: self.image_test, self.y : self.label_test})
        
        self.train_loss.append(loss)
        self.valid_loss.append(valid_loss)
        self.train_accuracy.append(acc)
        self.test_accuracy.append(test_acc)
        self.decisions = decisions

    def stop(self):
        self.session.close()
        tf.reset_default_graph()
        

#======== FUNCTIONS TO VISUALIZE NEURAL NETWORK ========#
def conv_to_img(v,ix,iy,ch,cy,cx, p = 0):
    '''Convert Output of convolutional layer to image'''
    v = np.reshape(v,(iy,ix,ch))
    ix += 2
    iy += 2
    npad = ((1,1), (1,1), (0,0))
    v = np.pad(v, pad_width=npad, mode='constant', constant_values=p)
    v = np.reshape(v,(iy,ix,cy,cx)) 
    v = np.transpose(v,(2,0,3,1)) #cy,iy,cx,ix
    v = np.reshape(v,(cy*iy,cx*ix))
    return v

def array_to_img(array):
    '''Convert output of fully connected layer to image'''
    shape = len(array)
    img = []
    for num in array:
        img.append(np.ones(shape*shape)*num)
    img = np.asarray(img)
    img = img.reshape((shape*shape,shape))
    img = np.transpose(img)
    return img

def weights_to_img(weights):
    '''Convert weights of layer to image'''
    shape = weights.shape
    img = np.asarray(weights).reshape(shape[0],shape[0]*shape[-1],shape[-2])
    img = img.reshape(img.shape[0]*img.shape[-1],img.shape[-2])
    return img

        
class Viewer(widgets.VBox):
    """GUI."""
    
    def __init__(self, controller):
        """Constructor"""
        super().__init__()
        
        # Save pointer to controller
        self.controller = controller
                  
        # GUI elements
        self.progress = widgets.FloatProgress(value=0, min=0, max=100)
        self.output = widgets.Label(value="Hello.")
        
        self.lr = widgets.BoundedFloatText(value=0.5, min=0, max=1.0, step=0.01, description='Learning Rate:')
        self.batch_size = widgets.IntText(value=2, min=0, max=1000, description='Batch Size:') # 20
        self.epochs = widgets.IntText(value=2, min=0, max=1000, description="Epochs:") # 100
        
        # Buttons
        self.playpause_button = widgets.Button(description='Play', button_style='success', icon='play')
        self.stop_button = widgets.Button(description='Stop', button_style='danger', icon='stop', disabled=True)
        
        # Button callbacks
        def on_button_clicked_playpause(b):
            
            if b.icon=='play':
                if not self.controller.is_alive():
                    self.controller.starting() # first time
                else:
                    self.controller.resume() # all the other times
                
                self.stop_button.disabled = True
                b.icon = 'pause'; b.button_style='warning'; b.description = 'Pause'
            else:
                self.controller.pause()
                b.icon='play'; b.button_style='success'; b.description='Play';
                self.stop_button.disabled = False
            
            # Deactivate input fields
            self.lr.disabled = True
            self.batch_size.disabled = True

        def on_button_clicked_stop(b):
            self.playpause_button.icon='play'; self.playpause_button.button_style='success'; self.playpause_button.description='Play';
            b.disabled = True
            self.controller.stop()
            
            # Activate input fields
            self.lr.disabled = False
            self.batch_size.disabled = False
        
        self.playpause_button.on_click(on_button_clicked_playpause)
        self.stop_button.on_click(on_button_clicked_stop)
               
        
        # Figures
        self.output_plot_cost = widgets.Output(); self.output_plot_train = widgets.Output(); self.output_plot_test = widgets.Output()
        self.tab = widgets.Tab(); self.tab.children = [self.output_plot_cost, self.output_plot_train, self.output_plot_test]
        self.tab.set_title(0, 'Cost functions'); self.tab.set_title(1, 'Training Progress'); self.tab.set_title(2, 'Test Classification')
        
        with self.output_plot_cost:
            self.fig_cost, self.axs_cost = plt.subplots(2,1, figsize=(8,6))
            self.fig_cost.suptitle('Cost Functions', fontsize=16)
        
        with self.output_plot_test:
            self.fig_test, self.axs_test = plt.subplots(4,6, figsize=(10,6))
            self.fig_test.suptitle('Classification on Test Dataset', fontsize=16)
            self.fig_test.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        with self.output_plot_train:
            self.fig_train, axs = plt.subplots(ncols=2, nrows=5)
            gs = axs[2, 1].get_gridspec()
            # remove the underlying axes
            for ax in axs[2:, 1]:
                ax.remove()
            axs[2:, 1] = self.fig_train.add_subplot(gs[2:, 1])
            self.axs_train = axs
            self.fig_train.suptitle('Training Convolutional Neural Network',fontsize=16)
        
        self.axs_train_title = ['Input','Activation after 1st Conv. Layer','Activation after 2nd Conv. Layer','Activation after Fully-connected Layer', 'Activation after Softmax',
                    'Input', 'Weights 1st Conv. Layer', 'Weights 2nd Conv. Layer']
        
        # Grid
        self.children = [self.output,
                         widgets.HBox([self.lr, self.batch_size]),
                         widgets.HBox([self.playpause_button, self.stop_button, self.progress]),
                         self.tab
                        ]    
        
    def update_text(self, i, training_iters, train_loss, train_accuracy, test_accuracy):
        self.output.value = 'Batch {}/{}: Loss={:5f}, Accuracy={:.5f}, Testing Accuracy={:.5f}'.format(i, training_iters, train_loss, train_accuracy, test_accuracy)
        return
        
    def plot_test(self):
        """Visualize Testing"""
        network = self.controller.network # TODO
        three_red = np.repeat('r', 3)
        elements = np.random.randint(len(network.image_test),size=20)
        
        for i,ax in enumerate(self.axs_test.flatten()):
            j = int(i/2)
            if i%2 == 0: # Image plot
                data = np.squeeze(network.image_test[elements[j]])
                if not ax.images: # Create image plot
                    ax.imshow(data, cmap='gray')
                    ax.set_xticks([]); ax.set_yticks([]);
                else: # update image plot
                    ax.images[0].set_array(data)
                    
            else: # Bar plot
                x_data = sorted(network.decisions[elements[j]])[-3:]
                y_true = int(np.argmax(network.label_test[elements[j]]))
                y_data = np.argpartition(network.decisions[elements[j]], -3)[-3:]
                if not ax.containers:                
                    ax.set_xticks([]); ax.set_yticks(np.arange(3)); 
                else:
                    ax.containers[0].remove()
                
                ax.set_yticklabels(y_data);
                colors = three_red; colors[y_data == y_true] = 'g'
                ax.barh(np.arange(3), x_data, align='center', color=colors)
                
        plt.show()
    
        
    def plot_cost(self):
        """Visualize cost"""
        
        network = self.controller.network # TODO
        if len(network.train_accuracy) == 1:
            return
        
        # Accuracy
        ax = self.axs_cost[0]
        if not ax.lines: # decorate
            ax.plot(np.arange(len(network.train_accuracy)), network.train_accuracy)
            ax.plot(np.arange(len(network.test_accuracy)), network.test_accuracy)
            ax.set_xlabel('Batch Number'); ax.set_ylabel('Percentages');
            ax.legend(['Training Accuracy','Test Accuracy'])
            
        else: # update lines only
            ax.lines[0].set_data(np.arange(len(network.train_accuracy)), network.train_accuracy)
            ax.lines[1].set_data(np.arange(len(network.test_accuracy)), network.test_accuracy)
            ax.relim(); ax.autoscale_view(True,True,True); # update limits
            
        # Loss
        ax = self.axs_cost[1]
        if not ax.lines: # decorate
            ax.plot(np.arange(len(network.train_loss)), network.train_loss)
            ax.plot(np.arange(len(network.valid_loss)), network.valid_loss)
            ax.set_xlabel('Batch Number'); 
            ax.legend(['Training Loss','Valid Loss'])
            
        else: # update lines only
            ax.lines[0].set_data(np.arange(len(network.train_loss)), network.train_loss)
            ax.lines[1].set_data(np.arange(len(network.valid_loss)), network.valid_loss)
            ax.relim(); ax.autoscale_view(True,True,True); # update limits

        plt.show()
        
    def plot_train(self):
        """Plot training layers"""
        
        layers = self.controller.network.layers
        if not layers:
            return
        
        axs = self.axs_train
        if not axs[0,0].lines: # decorate everything
            for i,ax in enumerate(axs.T.flatten()[0:-2]):                
                ax.imshow(layers[i], cmap='gray')
                ax.set_title(self.axs_train_title[i])
                ax.set_xticks([]); ax.set_yticks([]);
                
                if i > 5:
                    for k in range(int(layers[i].shape[0]/5)):
                        ax.axhline(k*5-0.5, color='yellow')
                    for k in range(int(layers[i].shape[1]/5)):
                        ax.axvline(k*5-0.5, color='yellow')
        else: # update only images
            for i,ax in enumerate(axs.T.flatten()[0:-2]):
                ax.images[0].set_array(layers[i])
    
    def update_progress(self, i, i_max):
        if self.progress.value == 100:
            self.progress.value = 0
        self.progress.value = i / i_max*100
                
    def stop(self):
        self.output.value = "Stopped"
        self.progress.value = 0
        
        for ax in np.concatenate((self.axs_cost.flatten(), self.axs_train.flatten(), self.axs_test.flatten())):
            ax.clear()
    
    def close(self):
        self.stop()
        self.progress.close()
        self.playpause_button.disabled = True
        self.output.value = "Closed"      


class ControllerThread(threading.Thread):
    """Threaded controller."""
    # copied from     https://gist.github.com/the-moog/94b09b49232731bd2a3cedd24501e23b 
    # and merged with https://stackoverflow.com/questions/33640283/thread-that-i-can-pause-and-resume
    
    def __init__(self):
        """Constructor"""
        super().__init__() # init parent
        
        # Update
        self.update_plot_train = 20
        self.update_plot_test = 10
        self.update_plot_cost = 5
        
        # Events (Thread booleans)
        self._quit = threading.Event() 
        self._is_running = threading.Event() 
        self._thing_done = threading.Event()
        self._thing_done.set() # set True
        self._stopped = threading.Event()
        self._stopped.set() # set True
                        
        # Model, Viewer, Controller (MVC):
        # GUI
        self.gui = self.gui = Viewer(self)
        
        # Model
        self.network = MnistCnn()
        #self.network.init_model(self.gui.lr.value, self.gui.batch_size.value, self.gui.epochs.value)
        
                
    def run(self):
        """Main working routine."""
        
        i = 0 # count training iters
        j = 0 # count epochs
        while not self._quit.isSet():
            self._is_running.wait() # if paused, flag is False. wait() wait's until it's True again
            if self._stopped.isSet():
                i = 0; j = 0;
                self._stopped.clear()
                
            try:
                self._thing_done.clear() # set False
                
                # Update widgets function
                # has to be queued to main thread via IO Loop
                def update_progress_step(i=i):
                    if self._quit.isSet():
                        return
                    
                    # Update progress bar # TODO: still needed?
                    self.gui.update_progress(i, self.network.training_iters)
                    
                    # Update text output
                    self.gui.update_text(i, self.network.training_iters, self.network.train_loss[-1], self.network.train_accuracy[-1], self.network.test_accuracy[-1])
                    
                    # Update GUI
                    if i%self.update_plot_cost == 0:
                        self.gui.plot_cost()
                    if i%self.update_plot_test == 0:
                        self.gui.plot_test()
                    if i%self.update_plot_train == 0:
                        self.gui.plot_train()
                        
                    
                # work work work work work
                detail_flag = not (i%self.update_plot_train)
                self.network.train_model_step(i, detail_flag)
                    
            
                # propagate back to main loop
                # update widgets here
                ioloop.add_callback(update_progress_step)

                # increment counter
                if i <= self.network.training_iters:
                    i = i+1 
                else:
                    i = 0; j = j+1
                                    
                if j > self.network.epochs:
                    return
                
            finally:
                self._thing_done.set() # set True

    
    def quit(self):
        self._quit.set() # set True
        time.sleep(1)
        self.gui.close()
                
    def pause(self):
        self._is_running.clear() # set False
        self._thing_done.wait()

    def resume(self):
        if self._stopped.isSet():
            self.network.init_model(self.gui.lr.value, self.gui.batch_size.value, self.gui.epochs.value)
        
        self._is_running.set() # set True
        
    def stop(self):
        self.pause()
        self._stopped.set()
        
        time.sleep(1)
        self.network.stop()
        self.gui.stop()
    
    def starting(self):
        self.resume() # has to be called before start (because start means that everything has to be set up before the thread starts)
        self.start() #calls run function
        

# Create controller
thread = ControllerThread()
thread.gui

<div class="alert rwth-feedback">

    
# Feedback:

Liebe TeilnehmerInnen,

Wir würden uns freuen, wenn ihr am Ende jeder Aufgabe kurz eure Meinung aufschreibt. Ihr könnt auf die dadrunter liegende Zelle zu greifen und eure Anmerkungen zu der Aufgabe (oder auch generelles) reinschreiben.


</div>

In [ ]:
rwth_feedback.rwth_feedback('Feedback V6.3', [
    {'id': 'likes', 'type': 'free-text', 'label': 'Das war gut:'}, 
    {'id': 'dislikes', 'type': 'free-text', 'label': 'Das könnte verbessert werden:'}, 
    {'id': 'misc', 'type': 'free-text', 'label': 'Was ich sonst noch sagen möchte:'}, 
    {'id': 'learning', 'type': 'scale', 'label' : 'Ich habe das Gefühl etwas gelernt zu haben.'},
    {'id': 'supervision', 'type': 'scale', 'label' : 'Die Betreuung des Versuchs war gut.'},
    {'id': 'script', 'type': 'scale', 'label' : 'Die Versuchsunterlagen sind verständlich.'},
], "feedback.json", 'pti@ient.rwth-aachen.de')